<a href="https://colab.research.google.com/github/zeruel91/DataAnalysis/blob/main/Data_ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
#경고 비표시
import warnings
warnings.filterwarnings('ignore')

In [63]:
import pandas as pd
uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [64]:
kokyaku_data = pd.read_excel('kokyaku_daicho.xlsx')
kokyaku_data.head()
#데이터에 입력 오류 또는 표기방법에 차이가 부정합을 일으킬 때 이것을
#데이터의 정합성에 문제가 있다.
#(결측치가 있다.) 

# 예를 들면 홍길동/홍 길동/홍 길 동/홍길 동
#인간의 데이터의 오류를 보완하면서 데이터를 이해하지만 시스템은 그렇지 않다.
#데이터의 오류를 해소하고 정합성을 보장하는 것은 데이터 분석의 
#시작이며 아주중요한 부분이다.


,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


In [65]:
uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [66]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

In [67]:
#상품의 표기법과 가격의 NaN처리 필요 확인.
#상품A/ 상 품 a / 상품a 를 각기 다른 상품으로 이해.

In [68]:
uriage_data["purchase_date"]=pd.to_datetime(uriage_data["purchase_date"])
uriage_data["purchase_month"]=uriage_data["purchase_date"].dt.strftime('%Y%m')
res = uriage_data.pivot_table(index = "purchase_month",
                              columns="item_name",aggfunc='size',fill_value = 0)

In [69]:
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,2,0,1,0,0,0,0,0,...,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,...,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,...,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0


In [70]:
res = uriage_data.pivot_table(index = "purchase_month",columns="item_name",
                              values = "item_price"
                              ,aggfunc='sum',fill_value = 0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,1500,1600,2000,0,0,4800,0,1900,0,0,0,0,0,...,6000,6500,7800,7000,10400,13500,16000,16500,19200,19500,21000,27000,20800,25500,36000,28500,28000,12600,41800,27600,24000,20000,0,0,300,0,0,0,0,0,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,0,0,0,0,0,0,0,0,400,0,0,0,0,900,0,0,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7600,5500,6600,7000,12800,8100,11000,14300,13200,16900,23800,25500,30400,30600,32400,32300,40000,37800,39600,43700,31200,20000,2600,0,0,0,0,0,0,1000,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6000,4000,11400,7700,14400,10800,13000,14300,21600,26000,16800,15000,32000,34000,27000,26600,22000,46200,35200,34500,38400,35000,0,0,0,400,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,2000,2200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1500,0,0,0,0,0,7500,0,0,0,0,1200,0,...,7200,8000,7800,9100,7200,14400,13000,12100,15600,14300,22400,18000,22400,28900,30600,28500,28000,16800,19800,20700,38400,32500,0,0,0,0,500,700,0,0,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,0,0,0,100,0,1900,100,0,0,0,0,0,0,0,1100,1300,0,0,0,0,0,2200,0,0,0,0,0,0,600,0,...,5200,9000,9000,14000,12000,13500,8000,15400,12000,20800,23800,24000,28800,15300,32400,20900,26000,33600,8800,36800,31200,45000,0,0,300,0,0,0,0,0,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,0,0,0,0,0,0,0,300,0,0,0,700,0,0,0,0,0,1500,0,0,2100,0,0,0,0,100,0,0,0,800,...,7200,6500,9600,7700,8800,12600,20000,15400,14400,19500,15400,19500,24000,23800,27000,34200,22000,29400,33000,25300,24000,30000,0,0,0,0,0,0,900,0,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1700,0,0,0,0,500,0,0,800,0,0,0,0,1500,0,0,0,2200,0,0,0,0,0,500,0,0,...,6000,4500,9600,11900,12800,13500,17000,9900,20400,13000,18200,25500,38400,18700,25200,38000,34000,23100,52800,32200,38400,27500,0,0,0,0,0,0,900,0,0,0,1500,0,1800,0,0,0,0,0


In [71]:
uriage_data["item_name"]=uriage_data["item_name"].str.upper()
uriage_data["item_name"]=uriage_data["item_name"].str.replace("  ","")
uriage_data["item_name"]=uriage_data["item_name"].str.replace(" ","")
uriage_data.sort_values(by=["item_name"]).ascending = True
#대문자로 바꾸고 공백 제거.

In [72]:
uriage_data["item_name"].head()

0    상품A
1    상품S
2    상품A
3    상품Z
4    상품A
Name: item_name, dtype: object

In [73]:
print(pd.unique(uriage_data["item_name"]))
print(len(pd.unique(uriage_data["item_name"])))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


In [74]:
#금액의 결측치를 수정하자.
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [75]:
#결측치에 따라서 현장에서 조사후 수정하거나 추가적으로 채워넣는 등의 여러가지 대응이 가능하다.

In [76]:
flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null,"item_name"].unique()):
  price = uriage_data.loc[(~flg_is_null)&(uriage_data["item_name"]==trg),"item_price"].max()
  uriage_data["item_price"].loc[(flg_is_null)&(uriage_data["item_name"]==trg)] = price
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [77]:
#1행에서 아이템프라이쓰 중에 결측치가 있응 곳을조사, 그결과를 flg is null에 저장
#결측지가 있는 상푸몀ㅇ 리스트 작성
#리스트는 변수의 값을 리스트로 변환
#loc는 는 조건에 일치하는 데이터 추출
#여기서는 조건이 '금액에 결측치가 있다.'
#아이템 네임은 어떤 칼럼을 가져올지 지정
#결측치가 존재하는 상품명을 가져온다
#유니크는 셋 옵션과 같음. 중복된 상품명 표시 X

for trg in list(uriage_data["item_name"].sort_values().unique()):
  print(trg+"의 최고가 : "+str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].max())
        + " , 최저가 : "+str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].min(skipna=False)))
#1행에서 모든 상품에 대해 반복문 적용. 상품에 설정된 최대,최소 금액을 출력
#결과로 모든 사품의 최대,최소 금액이 일치하는 것으로 봐서 성공적으로 금액을 수정한것을 "검증"


상품A의 최고가 : 100.0 , 최저가 : 100.0
상품B의 최고가 : 200.0 , 최저가 : 200.0
상품C의 최고가 : 300.0 , 최저가 : 300.0
상품D의 최고가 : 400.0 , 최저가 : 400.0
상품E의 최고가 : 500.0 , 최저가 : 500.0
상품F의 최고가 : 600.0 , 최저가 : 600.0
상품G의 최고가 : 700.0 , 최저가 : 700.0
상품H의 최고가 : 800.0 , 최저가 : 800.0
상품I의 최고가 : 900.0 , 최저가 : 900.0
상품J의 최고가 : 1000.0 , 최저가 : 1000.0
상품K의 최고가 : 1100.0 , 최저가 : 1100.0
상품L의 최고가 : 1200.0 , 최저가 : 1200.0
상품M의 최고가 : 1300.0 , 최저가 : 1300.0
상품N의 최고가 : 1400.0 , 최저가 : 1400.0
상품O의 최고가 : 1500.0 , 최저가 : 1500.0
상품P의 최고가 : 1600.0 , 최저가 : 1600.0
상품Q의 최고가 : 1700.0 , 최저가 : 1700.0
상품R의 최고가 : 1800.0 , 최저가 : 1800.0
상품S의 최고가 : 1900.0 , 최저가 : 1900.0
상품T의 최고가 : 2000.0 , 최저가 : 2000.0
상품U의 최고가 : 2100.0 , 최저가 : 2100.0
상품V의 최고가 : 2200.0 , 최저가 : 2200.0
상품W의 최고가 : 2300.0 , 최저가 : 2300.0
상품X의 최고가 : 2400.0 , 최저가 : 2400.0
상품Y의 최고가 : 2500.0 , 최저가 : 2500.0
상품Z의 최고가 : 2600.0 , 최저가 : 2600.0


In [78]:
kokyaku_data["고객이름"].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [79]:
# kokyaku_data["고객이름"]=kokyaku_data["고객이름"].str.upper()
kokyaku_data["고객이름"]=kokyaku_data["고객이름"].str.replace("  ","")
kokyaku_data["고객이름"]=kokyaku_data["고객이름"].str.replace(" ","")
kokyaku_data["고객이름"].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

In [80]:
#날짜 오류를 수정하자!
kokyaku_data["등록일"]

0      2018-01-04 00:00:00
1                    42782
2      2018-01-07 00:00:00
3                    42872
4                    43127
              ...         
195    2017-06-20 00:00:00
196    2018-06-20 00:00:00
197    2017-04-29 00:00:00
198    2019-04-19 00:00:00
199    2019-04-23 00:00:00
Name: 등록일, Length: 200, dtype: object

In [81]:
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit() #숫자로 되있는 데이터들만 저장.
flg_is_serial.sum() # 숫자로 되어있는 데이터의 갯수.

22

In [82]:
# import datetime
# kokyaku_data["등록일"] = str(datetime.timedelta(seconds=kokyaku_data["등록일"]))

fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial,"등록일"]
                             .astype("float"),unit = "D")+pd.to_datetime("1900/01/01")
fromSerial


1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [83]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial,"등록일"]) #숫자가 아니었던 놈들.
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [84]:
kokyaku_data["등록일"] = pd.concat([fromSerial,fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [85]:
kokyaku_data["등록연월"] = kokyaku_data["등록일"].dt.strftime('%Y%m') #연월만 표시. dt.strftime()

kokyaku_data

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-18,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-19,201705
4,김시온,E시,2018-01-29,201801
...,...,...,...,...
195,김재희,G시,2017-06-20,201706
196,김도영,E시,2018-06-20,201806
197,김이안,F시,2017-04-29,201704
198,김시현,H시,2019-04-19,201904


In [89]:
kokyaku_data[['등록일','등록연월']].head()
kokyaku_data.groupby('등록연월').count()

,고객이름,지역,등록일
등록연월,,,
201701,15,15,15
201702,11,11,11
201703,14,14,14
201704,15,15,15
201705,13,13,13
201706,14,14,14
201707,17,17,17
201801,13,13,13
201802,15,15,15
